In [5]:
# from model import create_eva_vit_g
# model = create_eva_vit_g(precision="fp16").cuda()
# import torch
# with torch.cuda.amp.autocast():
#     inputs = torch.randn(5, 3, 224, 224).cuda()
#     result = model(inputs)
    



In [2]:
import json
from box import Box

with open('/home/huimingsun/Desktop/NGP/video_sgements/config/config.json', 'r') as f:
    config_dict = json.load(f)



config = Box(config_dict)

# Now you can access values in the config like this:
vit_model = config.model.vit_model
vit_model_path = config.model.vit_model_path
# ... and so on for the other values


In [6]:


import torch
from torch.cuda.amp import autocast as autocast
import torch.nn as nn
from eva_vit import Trans_Block

try:
    from .blip2 import Blip2Base, disabled_train
except:
    from blip2 import Blip2Base, disabled_train


class VSeg(Blip2Base):
    """
    VideoChat model.
    """
    def __init__(self, config):
        super().__init__()


        self.tokenizer = self.init_tokenizer()
        self.low_resource = config.low_resource

        self.vit_precision = config.vit_precision
        print(f'Loading VIT. Use fp16: {config.vit_precision}')
        self.visual_encoder, self.ln_vision = self.init_vision_encoder(
            config.vit_model, config.img_size, config.drop_path_rate, 
            config.use_grad_checkpoint, config.vit_precision, config.vit_model_path,
            temporal_downsample=config.temporal_downsample,
            no_lmhra=config.no_lmhra, 
            double_lmhra=config.double_lmhra,
            lmhra_reduction=config.lmhra_reduction, 
            gmhra_layers=config.gmhra_layers, 
            gmhra_drop_path_rate=config.gmhra_drop_path_rate,
            gmhra_dropout=config.gmhra_dropout, 
        )
        if config.freeze_vit:
            print("freeze vision encoder")
            if not config.freeze_mhra:
                open_list = []
                for name, param in self.visual_encoder.named_parameters():
                    if 'mhra' not in name:
                        param.requires_grad = False
                    else:
                        open_list.append(name)

            else:
                for name, param in self.visual_encoder.named_parameters():
                    param.requires_grad = False
                self.visual_encoder = self.visual_encoder.eval()
                self.visual_encoder.train = disabled_train
                for name, param in self.ln_vision.named_parameters():
                    param.requires_grad = False
                self.ln_vision = self.ln_vision.eval()
                self.ln_vision.train = disabled_train
            
        self.blocks = nn.ModuleList([
                Trans_Block(dim = config.embedding_size, num_heads = config.embedding_size//88, mlp_ratio= 4.3637)
                    for i in range(10)])
        self.norm = nn.LayerNorm(config.embedding_size)
        self.fc_norm = nn.LayerNorm(config.embedding_size)

        self.head = nn.Linear(config.embedding_size, config.frames)
        self.score_head = nn.Linear(config.embedding_size, 1)
        self.max_txt_len = config.max_txt_len


    def vit_to_cpu(self):
        self.ln_vision.to("cpu")
        self.ln_vision.float()
        self.visual_encoder.to("cpu")
        self.visual_encoder.float()

    def forward_features(self, interval_1,interval_2):

        with self.maybe_autocast():
            T = interval_1.shape[1]
            # use_image = True if T == 1 else False
            interval_1 = interval_1.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
            interval_2 = interval_2.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
            interval1_embeds = self.ln_vision(self.visual_encoder(interval_1))
            interval2_embeds = self.ln_vision(self.visual_encoder(interval_2))
        x = torch.concat((interval1_embeds, interval2_embeds), dim=1)
        return x


            
    def forward(self, interval_1,interval_2, y = None):
        
        x = self.forward_features(interval_1,interval_2)
        x = self.norm(x)
        for block in self.blocks:
            x = block(x)
        x = x[:, 0]
        x = self.head(x)
        if y!= None:
            loss_fn = nn.BCEWithLogitsLoss()
            loss = loss_fn(x, y)
            return x, loss
        else:
            return x     

In [2]:
import json
from box import Box

with open('/home/huimingsun/Desktop/NGP/video_sgements/config/config.json', 'r') as f:
    config_dict = json.load(f)

config = Box(config_dict)


In [3]:
model = VSeg(config.model).cuda()

Loading VIT. Use fp16: fp32
Temporal downsample: False
Load ViT model from: /home/huimingsun/Desktop/NGP/video_sgements/checkpoint/eva_vit_g.pth
Inflate: patch_embed.proj.weight, torch.Size([1408, 3, 14, 14]) => torch.Size([1408, 3, 1, 14, 14])
freeze vision encoder


In [4]:
video = torch.rand(1, 5, 3,  224, 224).cuda()
output = model(video,video)

output.shape

torch.Size([1, 8])

In [13]:
def encode_img(self, interval_1,interval_2):

    with self.maybe_autocast():
        T = interval_1.shape[1]
        # use_image = True if T == 1 else False
        interval_1 = interval_1.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]
        interval_2 = interval_2.permute(0, 2, 1, 3, 4) # [B,T,C,H,W] -> [B,C,T,H,W]

        interval1_embeds = self.ln_vision(self.visual_encoder(interval_1))
        interval2_embeds = self.ln_vision(self.visual_encoder(interval_2))

        print(interval1_embeds.shape, interval2_embeds.shape)
        
        
VSeg.encode_img = encode_img

In [32]:
nn.Linear(1408, 1)(torch.randn(1, 1286*2,1408)).shape

torch.Size([1, 2572, 1])

In [7]:

blocks = nn.ModuleList([
                Trans_Block(dim = 1408, num_heads = 1408//88, mlp_ratio= 4.3637)
                    for i in range(10)]).cuda()


In [8]:
x = torch.randn(1, 1286*2,1408).cuda()

In [9]:
for block in blocks:
    x = block(x)
print(x.shape)

torch.Size([1, 2572, 1408])


In [4]:
video = torch.rand(1, 5, 3,  224, 224).cuda()
model.encode_img(video,video)

feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 257, 1408])
feature map size torch.Size([5, 

In [29]:
import torch
from x_transformers import TransformerWrapper, Decoder

model = TransformerWrapper(
    num_tokens = 500,
    max_seq_len = 1024,
    attn_layers = Decoder(
        dim = 512,
        depth = 12,
        heads = 8
    )
).cuda()

x = torch.randint(0, 256, (1, 1024)).cuda()

model(x).shape # (1, 1024, 20000)

torch.Size([1, 1024, 500])

In [12]:
import torch
from recurrent_memory_transformer_pytorch import RecurrentMemoryTransformer

model = RecurrentMemoryTransformer(
    num_tokens = 20000,               # number of tokens
    num_memory_tokens = 128,          # number of memory tokens, this will determine the bottleneck for information being passed to the future
    dim = 512,                        # model dimensions
    depth = 6,                        # transformer depth
    causal = True,                    # autoregressive or not
    dim_head = 64,                    # dimension per head
    heads = 8,                        # heads
    seq_len = 2048,                   # sequence length of a segment
    use_flash_attn = True             # whether to use flash attention
).cuda()


In [13]:
x = torch.randint(0, 256, (1, 2048)).cuda()
# print(x.shape)
# x = torch.randint(0, 256, ( 197, 768))
# print(x.shape)
logits1, mem1, _ = model(x)        # (1, 1024, 20000), (1, 128, 512), None
print(logits1.shape, mem1.shape)
logits2, mem2, _ = model(x, mem1)  # (1, 1024, 20000), (1, 128, 512), None
logits3, mem3, _ = model(x, mem2)  # (1, 1024, 20000), (1, 128, 512), None

# and so on ...


torch.Size([1, 2048, 20000]) torch.Size([1, 128, 512])
